## 所有导入模块

In [1]:
import pandas as pd
import numpy as np
import pickle

## 读取数据文件

In [2]:
# 这里的.dat后缀是随便取的，保存的格式还是csv格式，所以调用read_csv就能继续用
Data_event = "sub22-005_events.csv"
Date_pupil = "sub22-005.dat"

In [2]:
# 这里的.dat后缀是随便取的，保存的格式还是csv格式，所以调用read_csv就能继续用
Data_event = "sub22-031_events.csv"
Date_pupil = "sub22-031.dat"

In [3]:
pupil_event = pd.read_csv("../Data/"+Data_event)

In [4]:
# 读取两个文件
pupil_data = pd.read_csv("../Data/"+Date_pupil)
pupil_event = pd.read_csv("../Data/"+Data_event)
# 将两个字符串转变为时间序列格式，方便处理
pupil_event["timestamp"] = pd.to_datetime(pupil_event["timestamp"])
pupil_data["timestamp"] = pd.to_datetime(pupil_data["timestamp"]) 

In [5]:
max_time = max(pupil_event[pupil_event.columns[0]])
min_time = min(pupil_event[pupil_event.columns[0]])

## 为pupil_data设置单层索引

In [6]:
pupil_data = pupil_data.set_index([pupil_data.columns[0]])

In [7]:
pupil_data

,pupil_size,pupil_x,pupil_y
timestamp,,,
2021-11-14 07:18:21.864,20,561.762,1364.700
2021-11-14 07:18:21.874,71,470.726,1328.900
2021-11-14 07:18:21.884,88,1060.800,843.747
2021-11-14 07:18:21.894,86,1060.550,840.337
2021-11-14 07:18:21.904,72,1064.030,831.979
...,...,...,...
2021-11-14 08:12:09.173,117,879.395,974.507
2021-11-14 08:12:09.183,27,660.148,1403.360
2021-11-14 08:12:09.193,58,664.001,1390.480


## 检查事件长度是否相同

In [8]:
def is_same_event_step(df):
    a = df[df[df.columns[1]].str.contains("TRIALID")].index
    event_step = a[1]-a[0]
    assert(np.diff(a) == (event_step)).all(),"The OneShot event Data have different step"
    print("Pass event step test")
    
    return event_step
    

In [9]:
event_step = is_same_event_step(pupil_event)

Pass event step test


## 为pupil_event设置多层索引???感觉不需要

通过了事件步长检查后才能进行多层索引设置

In [10]:
def set_event_id(df):
    trail_num = len(df[df[df.columns[1]].str.contains("TRIALID")].index)
    temp = np.ones(shape = (trail_num*event_step),dtype = int)
    for i in range(trail_num):
        temp[i*event_step:i*event_step+event_step] = temp[i*event_step:i*event_step+event_step] * (i+1)
    df["trail_id"] = temp
set_event_id(pupil_event)    
pupil_event

,timestamp,message,trail_id
0,2021-11-14 07:18:21.869,TRIALID 1,1
1,2021-11-14 07:18:23.370,!V TRIAL_VAR transitions stable,1
2,2021-11-14 07:18:23.871,!V TRIAL_VAR stake 1,1
3,2021-11-14 07:18:24.775,!E TRIAL_EVENT_VAR response_1,1
4,2021-11-14 07:18:25.879,!E TRIAL_EVENT_VAR monster_show,1
...,...,...,...
2211,2021-11-14 08:12:03.187,!E TRIAL_EVENT_VAR response_1,277
2212,2021-11-14 08:12:04.228,!E TRIAL_EVENT_VAR monster_show,277
2213,2021-11-14 08:12:05.147,!E TRIAL_EVENT_VAR response_2,277
2214,2021-11-14 08:12:06.230,!E TRIAL_EVENT_VAR reward,277


## pupilevent设置双索引

In [11]:
pupil_event

,timestamp,message,trail_id
0,2021-11-14 07:18:21.869,TRIALID 1,1
1,2021-11-14 07:18:23.370,!V TRIAL_VAR transitions stable,1
2,2021-11-14 07:18:23.871,!V TRIAL_VAR stake 1,1
3,2021-11-14 07:18:24.775,!E TRIAL_EVENT_VAR response_1,1
4,2021-11-14 07:18:25.879,!E TRIAL_EVENT_VAR monster_show,1
...,...,...,...
2211,2021-11-14 08:12:03.187,!E TRIAL_EVENT_VAR response_1,277
2212,2021-11-14 08:12:04.228,!E TRIAL_EVENT_VAR monster_show,277
2213,2021-11-14 08:12:05.147,!E TRIAL_EVENT_VAR response_2,277
2214,2021-11-14 08:12:06.230,!E TRIAL_EVENT_VAR reward,277


In [12]:
pupil_event = pupil_event.set_index(["trail_id","timestamp"])

In [13]:
pupil_event

message
trail_id timestamp                                               
1        2021-11-14 07:18:21.869                        TRIALID 1
         2021-11-14 07:18:23.370  !V TRIAL_VAR transitions stable
         2021-11-14 07:18:23.871             !V TRIAL_VAR stake 1
         2021-11-14 07:18:24.775    !E TRIAL_EVENT_VAR response_1
         2021-11-14 07:18:25.879  !E TRIAL_EVENT_VAR monster_show
...                                                           ...
277      2021-11-14 08:12:03.187    !E TRIAL_EVENT_VAR response_1
         2021-11-14 08:12:04.228  !E TRIAL_EVENT_VAR monster_show
         2021-11-14 08:12:05.147    !E TRIAL_EVENT_VAR response_2
         2021-11-14 08:12:06.230        !E TRIAL_EVENT_VAR reward
         2021-11-14 08:12:09.220                        TRIAL_END

[2216 rows x 1 columns]

In [14]:
# 提取index的所有值
pupil_event.index.levels[1]

DatetimeIndex(['2021-11-14 07:18:21.869000', '2021-11-14 07:18:23.370000',
               '2021-11-14 07:18:23.871000', '2021-11-14 07:18:24.775000',
               '2021-11-14 07:18:25.879000', '2021-11-14 07:18:26.423000',
               '2021-11-14 07:18:27.881000', '2021-11-14 07:18:30.869000',
               '2021-11-14 07:18:53.219000', '2021-11-14 07:18:54.720000',
               ...
               '2021-11-14 08:11:56.608000', '2021-11-14 08:11:59.598000',
               '2021-11-14 08:12:00.220000', '2021-11-14 08:12:01.722000',
               '2021-11-14 08:12:02.224000', '2021-11-14 08:12:03.187000',
               '2021-11-14 08:12:04.228000', '2021-11-14 08:12:05.147000',
               '2021-11-14 08:12:06.230000', '2021-11-14 08:12:09.220000'],
              dtype='datetime64[ns]', name='timestamp', length=2216, freq=None)

In [15]:
type(pupil_event.index.levels[1])

pandas.core.indexes.datetimes.DatetimeIndex

## 创建最终的数据格式

In [16]:
pupil_data

,pupil_size,pupil_x,pupil_y
timestamp,,,
2021-11-14 07:18:21.864,20,561.762,1364.700
2021-11-14 07:18:21.874,71,470.726,1328.900
2021-11-14 07:18:21.884,88,1060.800,843.747
2021-11-14 07:18:21.894,86,1060.550,840.337
2021-11-14 07:18:21.904,72,1064.030,831.979
...,...,...,...
2021-11-14 08:12:09.173,117,879.395,974.507
2021-11-14 08:12:09.183,27,660.148,1403.360
2021-11-14 08:12:09.193,58,664.001,1390.480


In [17]:
pupil_event

message
trail_id timestamp                                               
1        2021-11-14 07:18:21.869                        TRIALID 1
         2021-11-14 07:18:23.370  !V TRIAL_VAR transitions stable
         2021-11-14 07:18:23.871             !V TRIAL_VAR stake 1
         2021-11-14 07:18:24.775    !E TRIAL_EVENT_VAR response_1
         2021-11-14 07:18:25.879  !E TRIAL_EVENT_VAR monster_show
...                                                           ...
277      2021-11-14 08:12:03.187    !E TRIAL_EVENT_VAR response_1
         2021-11-14 08:12:04.228  !E TRIAL_EVENT_VAR monster_show
         2021-11-14 08:12:05.147    !E TRIAL_EVENT_VAR response_2
         2021-11-14 08:12:06.230        !E TRIAL_EVENT_VAR reward
         2021-11-14 08:12:09.220                        TRIAL_END

[2216 rows x 1 columns]

In [18]:
pupil_data

,pupil_size,pupil_x,pupil_y
timestamp,,,
2021-11-14 07:18:21.864,20,561.762,1364.700
2021-11-14 07:18:21.874,71,470.726,1328.900
2021-11-14 07:18:21.884,88,1060.800,843.747
2021-11-14 07:18:21.894,86,1060.550,840.337
2021-11-14 07:18:21.904,72,1064.030,831.979
...,...,...,...
2021-11-14 08:12:09.173,117,879.395,974.507
2021-11-14 08:12:09.183,27,660.148,1403.360
2021-11-14 08:12:09.193,58,664.001,1390.480


In [19]:
# 不能使用pickle.dump(pupil_data,"./pupil_data.pd")
# 需要直接调用pd自带的to_pickle
# 保存pupil_data
pupil_data.to_pickle("./pupil_data_31.pd")
# 保存pupil_event
pupil_event.to_pickle("./pupil_event_31.pd")